In [108]:
from datetime import datetime
import csv
import requests, bs4
import re
import pandas as pd
import os
from datetime import datetime
import dtale

In [22]:
def get_pbp(teams = [], year):
    
    # team will be a list
    # year will be an int
    
    # If no team is input, scrape for all teams
    # Need to have a switch for if scraping current season or not
    # First: Generate list of Game Log URLs
    
    for team in teams:
        team_games = get_urls(team, year)
    
#     for url in team_games:
        

SyntaxError: non-default argument follows default argument (<ipython-input-22-d5655b167c9d>, line 1)

In [23]:
def get_urls(team, year):
    
    # Team is a string, year is an int
    # Need a switch for current season
    
    # First page to start looking for games
    try:
        team_page = f'https://www.basketball-reference.com/teams/{team}/{year}_games.html'
        res = requests.get(team_page, 'html_parser')
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text, 'html.parser')
    except:
        'Page does not exist!'
        
    # Get just regular season games
    reg_season = soup.find('table',{'id':'games'})
    
    # Initialize list
    links = []
    
    # Compile URLs and add to list
    for entry in reg_season.find_all(attrs={'data-stat':'box_score_text'}):
        for link in entry.find_all('a'):
            links.append(link.get('href'))
            
    return {team: links}

In [24]:
def get_dates(links):
    
    dates = []
    for lst in links.values():
        for entry in lst:
            
            date = datetime.strptime(entry[11:19], '%Y%m%d').strftime('%m/%d/%Y')
            dates.append(date)

    return dates

In [25]:
links = get_urls('PHI',2019)

In [102]:
team_abbrevs = {
    'Cleveland Cavaliers': 'CLE',
    'Boston Celtics':'BOS',
    'Washington Wizards':'WAS',
    'Charlotte Hornets':'CHO',
    'Minnesota Timberwolves':'MIN',
    'Dallas Mavericks':'DAL',
    'Milwaukee Bucks':'MIL',
    'Philadelphia 76ers':'PHI',
    'Phoenix Suns':'PHO',
    'Los Angeles Lakers':'LAL',
    'Utah Jazz':'UTA',
    'Sacramento Kings':'SAC',
    'New York Knicks':'NYK',
    'New Orleans Pelicans':'NOP',
    'Detroit Pistons':'DET',
    'Atlanta Hawks':'ATL',
    'Chicago Bulls':'CHI',
    'Miami Heat':'MIA',
    'Memphis Grizzlies':'MEM',
    'Golden State Warriors':'GSW',
    'Denver Nuggets':'DEN',
    'Brooklyn Nets':'BRK',
    'Los Angeles Clippers':'LAC',
    'Portland Trail Blazers':'POR',
    'Indiana Pacers':'IND',
    'San Antonio Spurs':'SAS',
    'Houston Rockets':'HOU',
    'Oklahoma City Thunder':'OKC',
    'Toronto Raptors':'TOR',
    'Orlando Magic':'ORL'
    }

for team in team_abbrevs.values():
    team_links = get_urls(team, 2019)
    team_dates = get_dates(team_links)

CLE
BOS
WAS
CHO
MIN
DAL
MIL
PHI
PHO
LAL
UTA
SAC
NYK
NOP
DET
ATL
CHI
MIA
MEM
GSW
DEN
BRK
LAC
POR
IND
SAS
HOU
OKC
TOR
ORL


In [99]:
def get_box_scores(urls):
    
    # List of urls
    # This will be called immediately after the team URLs were scraped
    # Need team input to match box scores with correct output data columns
    # Box score output should always be the same structure:
        # Away Team
        # Away Q1
        # Away Q2
        # Away Q3
        # Away Q4
        # Away Final
        # Home Team
        # Home Q1
        # Home Q2
        # Home Q3
        # Home Q4
        # Home Final
    
    away = []
    away_q1 = []
    away_q2 = []
    away_q3 = []
    away_q4 = []
    away_OT = []
    away_final = []
    home = []
    home_q1 = []
    home_q2 = []
    home_q3 = []
    home_q4 = []
    home_OT = []
    home_final = []
    
    all_lists = [away, away_q1, away_q2, away_q3, away_q4, away_OT, away_final,
               home, home_q1, home_q2, home_q3, home_q4, home_OT, home_final]
    
    for entry in urls.values():
        for url in entry:
            current_game = f'https://www.basketball-reference.com{url}'
            res = requests.get(current_game, 'html_parser')
            res.raise_for_status()
            soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'lxml')

            line_score = soup.find('table',{'id':'line_score'})
            df = pd.read_html(str(line_score),flavor='bs4')[0]
            
            if 'OT' in df['Scoring'].columns:
                away.append(df['Unnamed: 0_level_0']['Unnamed: 0_level_1'].iloc[0])
                home.append(df['Unnamed: 0_level_0']['Unnamed: 0_level_1'].iloc[1])
                away_q1.append(df['Scoring']['1'].iloc[0])
                away_q2.append(df['Scoring']['2'].iloc[0])
                away_q3.append(df['Scoring']['3'].iloc[0])
                away_q4.append(df['Scoring']['4'].iloc[0])
                away_OT.append(df['Scoring']['OT'].iloc[0])
                away_final.append(df['Scoring']['T'].iloc[0])
                home_q1.append(df['Scoring']['1'].iloc[1])
                home_q2.append(df['Scoring']['2'].iloc[1])
                home_q3.append(df['Scoring']['3'].iloc[1])
                home_q4.append(df['Scoring']['4'].iloc[1])
                home_OT.append(df['Scoring']['OT'].iloc[1])
                home_final.append(df['Scoring']['T'].iloc[1])
                
            else:
                away.append(df['Unnamed: 0_level_0']['Unnamed: 0_level_1'].iloc[0])
                home.append(df['Unnamed: 0_level_0']['Unnamed: 0_level_1'].iloc[1])
                away_q1.append(df['Scoring']['1'].iloc[0])
                away_q2.append(df['Scoring']['2'].iloc[0])
                away_q3.append(df['Scoring']['3'].iloc[0])
                away_q4.append(df['Scoring']['4'].iloc[0])
                away_OT.append(0)
                away_final.append(df['Scoring']['T'].iloc[0])
                home_q1.append(df['Scoring']['1'].iloc[1])
                home_q2.append(df['Scoring']['2'].iloc[1])
                home_q3.append(df['Scoring']['3'].iloc[1])
                home_q4.append(df['Scoring']['4'].iloc[1])
                home_OT.append(0)
                home_final.append(df['Scoring']['T'].iloc[1])
            
    line_score_dict = {
        'home_team':home,
        'away_team':away,
        'away_q1':away_q1,
        'away_q2':away_q2,
        'away_q3':away_q3,
        'away_q4':away_q4,
        'away_OT':away_OT,
        'away_final':away_final,
        'home_q1':home_q1,
        'home_q2':home_q2,
        'home_q3':home_q3,
        'home_q4':home_q4,
        'home_OT':home_OT,
        'home_final':home_final
    }
    
    
    return pd.DataFrame(line_score_dict)

In [100]:
df = get_box_scores(links)

In [107]:
pd.concat([df,pd.DataFrame({'date':get_dates(links)})], axis=1)

home_team away_team  away_q1  away_q2  away_q3  away_q4  away_OT  \
0        BOS       PHI       21       21       24       21        0   
1        PHI       CHI       41       17       18       32        0   
2        PHI       ORL       32       20       34       29        0   
3        DET       PHI       28       36       30       26       12   
4        MIL       PHI       34       30       17       27        0   
5        PHI       CHO       28       34       23       18        0   
6        PHI       ATL       25       22       13       32        0   
7        TOR       PHI       26       27       33       26        0   
8        PHI       LAC       30       27       29       27        0   
9        PHI       DET       24       23       25       27        0   
10       BRK       PHI       21       26       28       22        0   
11       IND       PHI       29       21       28       22        0   
12       PHI       CHO       33       17       35       34       13   
13       MEM       PHI       26       37       17       22        4   
14       MIA       PHI       32       32       32       28        0   
15       ORL       PHI       30       22       35       19        0   
16       PHI       UTA       24       28       32       23        0   
17       CHO       PHI       42       21       19       28       12   
18       PHI       PHO       36       26       24       28        0   
19       PHI       NOP       23       34       30       33        0   
20       PHI       CLE       31       23       30       37        0   
21       BRK       PHI       25       29       35       38        0   
22       PHI       NYK       27       21       20       23        0   
23       PHI       WAS       19       27       24       28        0   
24       PHI       MEM       25       19       24       27        0   
25       TOR       PHI       29       20       28       25        0   
26       DET       PHI       28       28       33       28        0   
27       PHI       DET       29       18       29       26        0   
28       PHI       BRK       31       26       37       33        0   
29       PHI       IND       24       25       31       33        0   
..       ...       ...      ...      ...      ...      ...      ...   
52       SAC       PHI       26       26       27       29        0   
53       PHI       TOR       40       32       28       19        0   
54       PHI       DEN       23       33       26       28        0   
55       PHI       LAL       40       27       27       26        0   
56       PHI       BOS       28       24       25       35        0   
57       NYK       PHI       37       34       29       26        0   
58       PHI       MIA       21       29       25       27        0   
59       PHI       POR       27       32       41       30        0   
60       NOP       PHI       31       34       28       18        0   
61       OKC       PHI       37       23       26       22        0   
62       PHI       GSW       31       24       38       27        0   
63       PHI       ORL       34       30       18       24        0   
64       CHI       PHI       30       27       29       21        0   
65       HOU       PHI       21       23       21       26        0   
66       PHI       IND       25       34       11       19        0   
67       PHI       CLE       12       35       30       22        0   
68       PHI       SAC       29       29       31       25        0   
69       MIL       PHI       31       31       27       41        0   
70       CHO       PHI       31       31       23       33        0   
71       PHI       BOS       39       30       21       25        0   
72       ATL       PHI       41       27       32       27        0   
73       ORL       PHI       31       29       17       21        0   
74       PHI       BRK       23       35       24       28        0   
75       MIN       PHI       38       30       24       26        0   
76       DAL       PHI     

In [ ]:
url = f'https://www.basketball-reference.com/boxscores/pbp/201410290IND.html'
res = requests.get(url)
res.raise_for_status()
soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'lxml')
pbp = soup.find('table',{'id':'pbp'})
PHI = pd.read_html(str(pbp), flavor='bs4')[0]

In [93]:
current_game = f'https://www.basketball-reference.com/boxscores/201810230DET.html'
res = requests.get(current_game, 'html_parser')
res.raise_for_status()
soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'lxml')

line_score = soup.find('table',{'id':'line_score'})
df = pd.read_html(str(line_score),flavor='bs4')[0]

if 'OT' in df['Scoring'].columns:
    dct = {'away_team':df['Unnamed: 0_level_0']['Unnamed: 0_level_1'].iloc[0],
                  'home_team': df['Unnamed: 0_level_0']['Unnamed: 0_level_1'].iloc[1],
                  'away_q1': df['Scoring']['1'].iloc[0],
                  'away_q2': df['Scoring']['2'].iloc[0],
                  'away_q3': df['Scoring']['3'].iloc[0],
                  'away_q4': df['Scoring']['4'].iloc[0],
                  'away_OT': df['Scoring']['OT'].iloc[0],
                  'away_total': df['Scoring']['T'].iloc[0],
                  'home_q1': df['Scoring']['1'].iloc[1],
                  'home_q2': df['Scoring']['2'].iloc[1],
                  'home_q3': df['Scoring']['3'].iloc[1],
                  'home_q4': df['Scoring']['4'].iloc[1],
                  'home_OT': df['Scoring']['OT'].iloc[1],
                  'home_total': df['Scoring']['T'].iloc[1]}
else:
    dct = {'away_team':df['Unnamed: 0_level_0']['Unnamed: 0_level_1'].iloc[0],
                      'home_team': df['Unnamed: 0_level_0']['Unnamed: 0_level_1'].iloc[1],
                      'away_q1': df['Scoring']['1'].iloc[0],
                      'away_q2': df['Scoring']['2'].iloc[0],
                      'away_q3': df['Scoring']['3'].iloc[0],
                      'away_q4': df['Scoring']['4'].iloc[0],
                      'away_total': df['Scoring']['T'].iloc[0],
                      'home_q1': df['Scoring']['1'].iloc[1],
                      'home_q2': df['Scoring']['2'].iloc[1],
                      'home_q3': df['Scoring']['3'].iloc[1],
                      'home_q4': df['Scoring']['4'].iloc[1],
                      'home_total': df['Scoring']['T'].iloc[1]}

In [95]:
dct

{'away_team': 'PHI',
 'home_team': 'DET',
 'away_q1': 28,
 'away_q2': 36,
 'away_q3': 30,
 'away_q4': 26,
 'away_OT': 12,
 'away_total': 132,
 'home_q1': 27,
 'home_q2': 34,
 'home_q3': 26,
 'home_q4': 33,
 'home_OT': 13,
 'home_total': 133}